In order to use `NetTopologySuite` (NTS) you need to install the nuget package.

In [ ]:
#r "nuget:NetTopologySuite"

Set up `NtsGeometryServices` according to needs

In [ ]:
NetTopologySuite.NtsGeometryServices.Instance = new NetTopologySuite.NtsGeometryServices(
    // default CoordinateSequenceFactory
    NetTopologySuite.Geometries.Implementation.CoordinateArraySequenceFactory.Instance,
    // default precision model
    new NetTopologySuite.Geometries.PrecisionModel(),
    // default SRID
    -1, 
    /********************************************************************
     * Note: the following arguments are only valid for NTS >= v2.2
     ********************************************************************/
    // Geometry overlay operation function set to use (Legacy or NG)
    NetTopologySuite.Geometries.GeometryOverlay.NG,
    // Coordinate equality comparer to use (CoordinateEqualityComparer or PerOrdinateEqualityComparer)
    new NetTopologySuite.Geometries.CoordinateEqualityComparer());

Read the input geometry that is the source for the triangulation

In [ ]:
var rdr = new NetTopologySuite.IO.WKTReader(NetTopologySuite.NtsGeometryServices.Instance);
var input = rdr.Read(@"POLYGON ((0 -8.1270121, 14.4 -8.1270121, 14.4 -5.3202380000000007, 
7.3976450000000007 -5.3202380000000007, 7.3976450000000007 -5.735, 7.3970140788122256 -5.7478426958116806, 
7.3951273913648334 -5.7605617094421628, 7.3920031074893133 -5.7730345498372664, 7.3876713157472915 -5.7851410967256358, 
7.3821737336612436 -5.7967647574426264, 7.3755633059519417 -5.8077935897813937, 7.3679036946518526 -5.8181213800575424, 
7.3592686660049678 -5.8276486660049676, 7.3497413800575426 -5.8362836946518524, 7.3394135897813939 -5.8439433059519414, 
7.3283847574426266 -5.8505537336612434, 7.316761096725636 -5.8560513157472913, 7.3046545498372657 -5.8603831074893131, 
7.292181709442163 -5.8635073913648341, 7.26662 -5.866025, 6.6066199999999995 -5.866025, 
6.5937773041883192 -5.8653940788122245, 6.581058290557837 -5.8635073913648332, 6.5685854501627334 -5.8603831074893122, 
6.556478903274364 -5.85605131574729, 6.5448552425573734 -5.8505537336612434, 6.533826410218607 -5.8439433059519406, 
6.5234986199424583 -5.8362836946518524, 6.5139713339950331 -5.8276486660049667, 6.5053363053481483 -5.8181213800575415, 
6.4976766940480593 -5.8077935897813928, 6.4910662663387564 -5.7967647574426264, 6.4855686842527094 -5.7851410967256358, 
6.4812368925106876 -5.7730345498372664, 6.4781126086351666 -5.7605617094421628, 6.475595 -5.735, 
6.475595 -5.3202380000000007, 0 -5.3202380000000007, 0 -8.1270121))");

Setup the triangulation builder, add sites and get the triangles

In [ ]:
var builder = new NetTopologySuite.Triangulate.DelaunayTriangulationBuilder();
builder.SetSites(input);
var result = builder.GetTriangles(input.Factory);

You may want to use `ConformingDelaunayTriangulationBuilder`. In that case you have to set the constraints as well.

Filter results to those triangles that are within the input geometry.

In [ ]:
var filter = new NetTopologySuite.Algorithm.Locate.IndexedPointInAreaLocator(input);
var resultWithin = new List<NetTopologySuite.Geometries.Geometry>(result.Count);
foreach (var tri in result) {
    if (filter.Locate(tri.Centroid.Coordinate) != NetTopologySuite.Geometries.Location.Exterior)
        resultWithin.Add(tri);
        //System.Console.WriteLine(tri.AsText());
}

System.Console.WriteLine(input.Factory.BuildGeometry(resultWithin.ToArray()).AsText());